In [2]:
from utils import RenderJSON, fetch_all_items

import streamlit as st
from congress import Congress

SENATE = 'senate'
HOUSE = 'house'
BOTH = 'both'

PROPUBLICA_API_KEY = st.secrets["PROPUBLICA_API_KEY"];
CONGRESS_API_KEY = st.secrets["CONGRESS_API_KEY"];

congress = Congress(PROPUBLICA_API_KEY)

In [60]:
def validate_bill_id(bill_id=''):
    pattern = r'^(hr|s|hjres|sjres|hconres|sconres|hres|sres)(\d+)-(\d+)$'
    return bool(re.match(pattern, bill_id))

In [68]:
validate_bill_id(should_fail)

False

In [51]:
import re
should_fail = "adjourn-118"
should_pass = "hconres119-117"

BILL_TYPES = ['hr', 's', 'hjres', 'sjres', 'hconres', 'sconres', 'hres', 'sres']

pattern = r'^(' + '|'.join(BILL_TYPES) + r')(\d+)-(\d+)$'
print(pattern
)

if re.match(pattern, should_pass):
    print('works')
    # bill_type, bill_n, congress_number = re.split(pattern, should_pass)
    # print(bill_type, bill_n, congress_number)
if re.match(pattern, should_fail):
    print("opps")
if re.match(pattern, "hcores119-117"):
    print("opps2")

^(hr|s|hjres|sjres|hconres|sconres|hres|sres)(\d+)-(\d+)$
works


In [52]:
pattern == r'^(hr|s|hjres|sjres|hconres|sconres|hres|sres)(\d+)-(\d+)$'

True

In [47]:
re.split(pattern, should_pass)

['', 'hconres', '119', '117', '']

In [3]:
dir(congress.bills)

['BASE_URI',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'amendments',
 'apikey',
 'by_member',
 'cosponsors',
 'enacted',
 'fetch',
 'get',
 'http',
 'introduced',
 'passed',
 'recent',
 'related',
 'subjects',
 'upcoming',
 'updated',
 'vetoed']

In [7]:
bills_deets = congress.bills.get(bill_id='sres21', congress='118')
RenderJSON(bills_deets)

In [9]:
a, b = "sres21-118".split('-')
print(a,b)

sres21 118


In [28]:
def get_bill_deets(bill_id):
    bill, congress_n = bill_id.split('-')

    return Congress(PROPUBLICA_API_KEY).bills.get(bill, congress_n)

RenderJSON(get_bill_deets("sres21-118"))

In [8]:
bills_deets = congress.bills.get(bill_id='hr1123', congress='118')
RenderJSON(bills_deets)

Use raw congress API
- ref book https://colab.research.google.com/drive/1xdYRN16cOvYAOFayvTqc0n4N43ItA_dU#scrollTo=0HXdQeCT5Mt4
- docs https://api.congress.gov/#/bill/bill_summaries

In [5]:
import requests
import re

In [20]:
BILL_TYPES = ['hr', 's', 'hjres', 'sjres', 'hconres', 'sconres', 'hres', 'sres']


bill_id = "sres21-118"
bill, congress = bill_id.split('-')

bill_type, bill_n = re.split('(\d+)', bill)[:2]
print(bill_type in bill_types)
# print
url = f"https://api.congress.gov/v3/bill/{congress}/{bill_type}/{bill_n}/summaries?api_key={CONGRESS_API_KEY}"
response = requests.get(url)
if response.status_code == 200:
    json_data = response.json()

RenderJSON(json_data)


True


In [34]:
def get_bill_summaries_official(bill_id):
    '''Takes ProPublica bill_id ex "sres21-118"
    https://api.congress.gov/#/bill/bill_summaries
    '''
    bill_types = ['hr', 's', 'hjres', 'sjres', 'hconres', 'sconres', 'hres', 'sres']
    bill, congress = bill_id.split('-')
    bill_type, bill_n = re.split('(\d+)', bill)[:2]
    assert(bill_type in bill_types)
    url = f"https://api.congress.gov/v3/bill/{congress}/{bill_type}/{bill_n}/summaries?api_key={CONGRESS_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        json_data = response.json()
    else:
        print("Error retrieving bill summaries. Status code:", response.status_code)
    return json_data

In [23]:
bills = [
    "hr5129-117",
    "hres1119-117",
    "hr6386-117"
]

In [24]:
[re.split('(\d+)', bill)[:2] for bill in bills]

[['hr', '5129'], ['hres', '1119'], ['hr', '6386']]

In [35]:
bill_sums = [get_bill_summaries_official(bill) for bill in bills]


In [27]:
RenderJSON(bill_sums[0])

## Compare ProPub to Official

In [ ]:
bill_id = "hr5129-117"


In [32]:
# Official
RenderJSON(get_bill_summaries_official(bill_id))

In [31]:
# ProPub
RenderJSON(get_bill_deets(bill_id))

In [66]:
from bs4 import BeautifulSoup

def html_to_structured_text(html_string):
    # Create a BeautifulSoup object from the HTML string
    soup = BeautifulSoup(html_string, 'html.parser')

    # Extract the text and create a structured text string
    structured_text = ''
    for tag in soup.find_all():
        tag_name = tag.name
        if tag_name == 'p':
            structured_text += f'\n\n{tag.text}\n\n'
        elif tag_name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
            structured_text += f'\n\n{tag.text}\n{"=" * len(tag.text)}\n\n'
        elif tag_name == 'ul':
            for li in tag.find_all('li'):
                structured_text += f'* {li.text}\n'
        elif tag_name == 'ol':
            for li in tag.find_all('li'):
                index = li.find_previous_siblings('li')
                index = str(len(index) + 1)
                structured_text += f'{index}. {li.text}\n'
    if structured_text == '':
        return html_string
        
    return structured_text

string = "<p><b>Promoting United States International Leadership in 5G Act of 2021</b></p> <p>This bill directs the President to establish an interagency working group to provide assistance and technical expertise to enhance U.S. leadership in international bodies&nbsp;that set&nbsp;standards for 5th and future generations of mobile telecommunications (5G) technology, such as the International Telecommunication Union. The President shall also work to increase the participation of allies, partners, and the private sector. (International bodies develop standards for certain technologies, such as those used in wireless data transmission, to ensure interoperability between different companies' products; stakeholders, such as companies that manufacture equipment that uses such technologies or companies that hold relevant patents, typically participate in the standards-setting process.) </p> <p>The interagency working group must report to Congress on&nbsp;(1) its strategy to promote U.S. leadership in the standards-setting bodies relevant to 5G technology; (2) its strategy for diplomatic engagement with allies and partners to share security-risk information related to 5G; (3) its strategy for engagement with private sector communications and information service providers; and (4) China's activities in standards-setting bodies for 5G technology, including the scope and scale of such activities.</p>"
html_to_structured_text(string)

"\n\nPromoting United States International Leadership in 5G Act of 2021\n\n\n\nThis bill directs the President to establish an interagency working group to provide assistance and technical expertise to enhance U.S. leadership in international bodies\xa0that set\xa0standards for 5th and future generations of mobile telecommunications (5G) technology, such as the International Telecommunication Union. The President shall also work to increase the participation of allies, partners, and the private sector. (International bodies develop standards for certain technologies, such as those used in wireless data transmission, to ensure interoperability between different companies' products; stakeholders, such as companies that manufacture equipment that uses such technologies or companies that hold relevant patents, typically participate in the standards-setting process.) \n\n\n\nThe interagency working group must report to Congress on\xa0(1) its strategy to promote U.S. leadership in the standa

In [69]:
string = "<p>This concurrent resolution condemns the October 25, 2021, coup in Sudan. It recognizes the Prime Minister and his cabinet as the constitutional leaders of Sudan's transitional government and calls for Sudan's military junta, among other things, to</p> <ul> <li>immediately release all civilian government officials, civil society members, and other individuals detained in connection with the coup;</li> <li>return to constitutional rule under the transitional constitution; and </li> <li>lift the state of emergency, including complete restoration of all means of communication. </li> </ul> <p>Finally, the concurrent resolution calls on international partners to</p> <ul> <li> join U.S. efforts to impose targeted sanctions on the junta and other accomplices to the coup; and</li> <li>suspend Sudan's participation in all regional multilateral organizations until Sudan is returned to constitutional rule under the transitional constitution.</li> </ul>"

html_to_structured_text(string)


"\n\nThis concurrent resolution condemns the October 25, 2021, coup in Sudan. It recognizes the Prime Minister and his cabinet as the constitutional leaders of Sudan's transitional government and calls for Sudan's military junta, among other things, to\n\n* immediately release all civilian government officials, civil society members, and other individuals detained in connection with the coup;\n* return to constitutional rule under the transitional constitution; and \n* lift the state of emergency, including complete restoration of all means of communication. \n\n\nFinally, the concurrent resolution calls on international partners to\n\n*  join U.S. efforts to impose targeted sanctions on the junta and other accomplices to the coup; and\n* suspend Sudan's participation in all regional multilateral organizations until Sudan is returned to constitutional rule under the transitional constitution.\n"